# Introduction to the Python control system

This is the introductory example explaining the connection setup between Python and the **Term** Terminal Software running the **Thales** Workstation Software.

In [99]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import math
import cmath

from jupyter_utils import executionInNotebook, notebookCodeToPython

# Utility functions

First, two utility functions are declared.  
With the first function the complex impedance is output to the console. The second function measures an impedance spectrum from individual impedance measuring points.

In [100]:
def printImpedance(impedance):
    print("Impedance: " + str(abs(impedance)) + " ohm, " + str(cmath.phase(impedance)) + " rad")
    return

def spectrum(scriptHandle, lower_frequency, upper_frequency, number_of_points):
    log_lower_frequency = math.log(lower_frequency)
    log_upper_frequency = math.log(upper_frequency)
    log_interval_spacing = (log_upper_frequency - log_lower_frequency) / (number_of_points - 1)
    
    for i in range(number_of_points):
        current_frequency = math.exp(log_lower_frequency + log_interval_spacing * i)
        print("Frequency: " + str(current_frequency))
        printImpedance(scriptHandle.getImpedance(current_frequency))
        
    return

# Connect to the Term software

Before calling the following commands, the Term software must be started and the Thales start screen must be displayed, in which methods can be selected.

In this case the Term runs on the same computer as Python, so **"localhost"** is chosen here as IP address, but you could also specify the IP address of the computer in the network, on which the Term software is running and to which the workstation is connected via USB.

In [101]:
TARGET_HOST = "localhost"

if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    connectionSuccessful = zenniumConnection.connectToTerm(TARGET_HOST, "ScriptRemote")
    if connectionSuccessful:
        print("connection successfull")
    else:
        print("connection not possible")
        sys.exit()

connection successfull


# Initialize the application and start the remote software in Thales

If the connection to the term is successfully established, the [ThalesRemoteConnection]([**ThalesRemoteScriptWrapper**](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper)) object that manages the connection to the Thales software. This object is passed to the constructor of a new [ThalesRemoteScriptWrapper](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) object.

The [ThalesRemoteScriptWrapper](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper) class implements the commands as methods, which are available via [Remote2](http://zahner.de/pdf/Remote2.pdf).

In [ ]:
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()

# DC measurement

## Potentiostatic measurement

First, a voltage of 1.0 V is output potentiostatically as an example. For this, potentiostatic is set first. Then the potential is set and the potentiostat is switched on.

In [ ]:
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setPotential(1.0)
    zahnerZennium.enablePotentiostat()

After the potentiostat is switched on, voltage and current are measured several times in a loop.

In [104]:
    for i in range(5):
        print("Potential: " + str(zahnerZennium.getPotential()))
        print("Current: " + str(zahnerZennium.getCurrent()))

Potential: 0.9998677
Current: 9.974955e-09
Potential: 0.9998511
Current: 9.986098e-09
Potential: 0.9998537
Current: 9.986843e-09
Potential: 0.9998545
Current: 9.987165e-09
Potential: 0.9998519
Current: 9.987215e-09


## Galvanostatic measurement

After the potentiostatic measurement, galvanostatic measurement is performed with 20 nA.

In [110]:
    zahnerZennium.disablePotentiostat()
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
    zahnerZennium.setCurrent(20e-9)
    zahnerZennium.enablePotentiostat()

    for i in range(5):
        print("Potential: " + str(zahnerZennium.getPotential()))
        print("Current: " + str(zahnerZennium.getCurrent()))

Potential: 2.002193
Current: 1.997753e-08
Potential: 2.002225
Current: 1.997747e-08
Potential: 2.002216
Current: 1.997732e-08
Potential: 2.002214
Current: 1.997726e-08
Potential: 2.002321
Current: 1.997749e-08


# Impedance measurement

## Parameterization of the impedance measurement

For the impedance measurement, the measuring frequency, the measuring amplitude and the number of periods to be averaged are now set.

In [ ]:
    zahnerZennium.disablePotentiostat()
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setPotential(1.0)
    zahnerZennium.enablePotentiostat()
    zahnerZennium.setFrequency(2000)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setNumberOfPeriods(3)

## Execute the measurement

Since the potentiostat is still switched on from the DC measurement, the impedance spectrum is now measured at the set DC potential of 1 V.

<div class="alert alert-block alert-info">
<b>Note:</b> If the potentiostat is set to potentiostatic before the impedance measurement and is switched off, the measurement is performed at the open circuit voltage/potential.
</div>
    

In [94]:
    printImpedance(zahnerZennium.getImpedance())
    printImpedance(zahnerZennium.getImpedance(2000))
    printImpedance(zahnerZennium.getImpedance(2000, 10e-3, 3))

    spectrum(zahnerZennium, 1000, 2e5, 10)

Impedance: 547196.5719921864 ohm, -1.5520085541292792 rad
Impedance: 547359.5080392776 ohm, -1.5560504561507276 rad
Impedance: 545647.8168196039 ohm, -1.5475191385625247 rad
Frequency: 999.9999999999998
Impedance: 1093152.1840988107 ohm, -1.5541098610449533 rad
Frequency: 1801.64823065441
Impedance: 608360.9166284435 ohm, -1.5566447152716172 rad
Frequency: 3245.936347020167
Impedance: 338850.93422477087 ohm, -1.5534574506394268 rad
Frequency: 5848.035476425735
Impedance: 188709.49485651217 ohm, -1.5607648667952978 rad
Frequency: 10536.102768906649
Impedance: 105002.74031643174 ohm, -1.563571690590275 rad
Frequency: 18982.350911593698
Impedance: 58280.26877940767 ohm, -1.5677592736754413 rad
Frequency: 34199.51893353398
Impedance: 32164.67708916102 ohm, -1.553742631962074 rad
Frequency: 61615.50277583351
Impedance: 18050.88258922538 ohm, -1.5609074524754027 rad
Frequency: 111009.46155696237
Impedance: 10070.008084286725 ohm, -1.5695291973836976 rad
Frequency: 200000.00000000006
Impedanc

## Switch off potentiostat

After the measurement, the potentiostat is switched off and **the amplitude must be set to 0** again after the impedance measurement.

In [ ]:
    zahnerZennium.disablePotentiostat()
    zahnerZennium.setAmplitude(0)

# Close the connection

In [111]:
    zenniumConnection.disconnectFromTerm()
    print("finish")

finish


# Deployment of the source code

**The following instruction is not needed by the user.**

It automatically extracts the pure python code from the jupyter notebook to provide it to the user. Thus the user does not need jupyter itself and does not have to copy the code manually.

The source code is saved in a .py file with the same name as the notebook.

In [97]:
    if executionInNotebook() == True:
        notebookCodeToPython("BasicIntroduction.ipynb")